In [1]:
import os.path as osp

import cv2
import matplotlib.cm as cm
import numpy as np
import torch.hub
import os
import model
from PIL import Image
from torchvision import transforms
from torchsummary import summary
from visualize.grad_cam import BackPropagation, GradCAM,GuidedBackPropagation

faceCascade = cv2.CascadeClassifier('./visualize/haarcascade_frontalface_default.xml')
shape = (48,48)
classes = [
    'Angry',
    'Disgust',
    'Fear',
    'Happy',
    'Sad',
    'Surprised',
    'Neutral'
]

In [5]:
def preprocess(image_path):
    transform_test = transforms.Compose([
        transforms.ToTensor()
    ])
    image = cv2.imread(image_path)
    # display(Image.fromarray(image))
    #它使用了detectMultiScale函数来对输入的图像进行人脸检测，并返回一个包含检测到的人脸位置和大小的矩形框列表
    faces = faceCascade.detectMultiScale(
        image,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(1, 1),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    if len(faces) == 0:
        print('no face found')
        face = cv2.resize(image, shape)
    else:
        (x, y, w, h) = faces[0]
        face = image[y:y + h, x:x + w]
        face = cv2.resize(face, shape)

    img = Image.fromarray(face).convert('L')
    # display(img)
    inputs = transform_test(img)
    return inputs, face

preprocess("../test/sad.jpg")

(tensor([[[0.6667, 0.6863, 0.6000,  ..., 0.5608, 0.5725, 0.5608],
          [0.6902, 0.7098, 0.2549,  ..., 0.5569, 0.5725, 0.5451],
          [0.6314, 0.2824, 0.1608,  ..., 0.5451, 0.5608, 0.5333],
          ...,
          [0.1216, 0.0941, 0.1176,  ..., 0.0745, 0.1412, 0.2157],
          [0.1569, 0.0980, 0.1059,  ..., 0.0863, 0.1490, 0.1804],
          [0.2000, 0.0980, 0.0627,  ..., 0.0980, 0.1059, 0.1373]]]),
 array([[[140, 183, 186],
         [153, 183, 188],
         [143, 157, 162],
         ...,
         [129, 149, 149],
         [133, 151, 150],
         [125, 150, 152]],
 
        [[148, 188, 190],
         [166, 186, 191],
         [ 60,  67,  69],
         ...,
         [128, 148, 149],
         [133, 151, 150],
         [123, 146, 148]],
 
        [[136, 171, 173],
         [ 65,  74,  81],
         [ 42,  40,  41],
         ...,
         [125, 145, 146],
         [130, 149, 147],
         [121, 142, 144]],
 
        ...,
 
        [[ 35,  29,  31],
         [ 22,  23,  38],


In [6]:
# 梯度归一化为8bit uint
def get_gradient_image(gradient):
    gradient = gradient.cpu().numpy().transpose(1, 2, 0)    #轴转置
    gradient -= gradient.min()
    gradient /= gradient.max()
    gradient *= 255.0
    return np.uint8(gradient)


# 图像融合
def get_gradcam_image(gcam, raw_image, paper_cmap=False):
    gcam = gcam.cpu().numpy()
    cmap = cm.jet_r(gcam)[..., :3] * 255.0  #使用cm.jet_r函数将gcam映射为颜色映射（colormap），并将其乘以255.0以将值范围映射到0-255之间
    if paper_cmap:
        alpha = gcam[..., None]
        gcam = alpha * cmap + (1 - alpha) * raw_image
    else:
        gcam = (cmap.astype(np.float64) + raw_image.astype(np.float64)) / 2
    return np.uint8(gcam)

In [ ]:
def guided_backprop(images, model_name):

    for i, image in enumerate(images):
        target, raw_image = preprocess(image['path'])
        image['image'] = target
        image['raw_image'] = raw_image

    net = model.Model(num_classes=len(classes))
    checkpoint = torch.load(os.path.join('../trained', model_name), map_location=torch.device('cpu'))
    net.load_state_dict(checkpoint['net'])              #加载名为model_name的训练模型参数
    net.eval()
    summary(net, (1, shape[0], shape[1]))               #做网络的概述

    result_images = []
    for index, image in enumerate(images):              
        img = torch.stack([image['image']])     
        bp = BackPropagation(model=net)
        probs, ids = bp.forward(img)
        gcam = GradCAM(model=net)
        _ = gcam.forward(img)

        gbp = GuidedBackPropagation(model=net)
        _ = gbp.forward(img)

        # Guided Backpropagation
        actual_emotion = ids[:,0]
        gbp.backward(ids=actual_emotion.reshape(1,1))
        gradients = gbp.generate()

        # Grad-CAM
        gcam.backward(ids=actual_emotion.reshape(1,1))
        regions = gcam.generate(target_layer='last_conv')

        # Get Images
        label_image = np.zeros((shape[0],65, 3), np.uint8)
        cv2.putText(label_image, classes[actual_emotion.data], (5, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)

        # 把probs显示为概率文本
        prob_image = np.zeros((shape[0],60,3), np.uint8)
        cv2.putText(prob_image, '%.1f%%' % (probs.data[:,0] * 100), (5, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        # 标准化图像(黑白)
        guided_bpg_image = get_gradient_image(gradients[0])
        guided_bpg_image = cv2.merge((guided_bpg_image, guided_bpg_image, guided_bpg_image))

        # 融合图像，regions=最后一次卷积
        grad_cam_image = get_gradcam_image(gcam=regions[0, 0],raw_image=image['raw_image'])

        # 融合图像???
        guided_gradcam_image = get_gradient_image(torch.mul(regions, gradients)[0])
        guided_gradcam_image = cv2.merge((guided_gradcam_image, guided_gradcam_image, guided_gradcam_image))

        img = cv2.hconcat([image['raw_image'],label_image,prob_image,guided_bpg_image,grad_cam_image,guided_gradcam_image])
        result_images.append(img)
        print(image['path'],classes[actual_emotion.data], probs.data[:,0] * 100)

    cv2.imwrite('../test/guided_gradcam.jpg',cv2.resize(cv2.vconcat(result_images), None, fx=2,fy=2))


def main():
    guided_backprop(
        images=[
            {'path': '../test/angry.jpg'},
            {'path': '../test/happy.jpg'},
            {'path': '../test/sad.jpg'},
            {'path': '../test/surprised.jpg'},
        ],
        model_name='private_model_233_66.t7'
    )